# A notebook for Checking Available Data
## Identify wells, tops, & curves that can be used in model

------------------------

## Goals for this notebook
- 1.A. Code to identify what tops are common enough to be included
- 1.B. Code to identify what curves are common enough to be included
- 1.C. Create list of wells that include necessary tops and curves based on two steps listed above
- 1.D Document what wells were not included in training and why!
- 1.E. Write to a file a initial dataset of wells to include and tops to include.
- 2.A. Split dataset in step above into 80/20 train/test subsets and label as such with a new column based on wells not rows
- 2.B There are many more "not near a pick" rows that at or near a pick rows. We'll need to create a column for what rows in the "away from pick" category to exclude in order to have balanced classes.

## THINGS YOU MIGHT NEED TO CHANGE IN THIS NOTEBOOK!
1. Links to the various files!
2. Decide what the minimum number of tops that need to be present before you can work with that top!
3. Decide what the minimum number of wells that have a curve name need to be before that curve is included as a required curve to include a well in the dataset you'll work with.

# Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import welly
from welly import Well
import lasio
import glob
from sklearn import neighbors
import pickle
import math
import dask
import dask.dataframe as dd
from dask.distributed import Client
from dask import delayed
from dask import compute
# import pdvega
# import vega
import dask.dataframe as dd
from dask.distributed import Client


/Users/justingosses/anaconda/envs/MannvilleDask2/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/justingosses/anaconda/envs/MannvilleDask2/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


'0.3.5'

In [2]:
print(welly.__version__)
print(dask.__version__)
print(pd.__version__)

0.3.5
0.18.2
0.23.3


In [3]:
%%timeit
import os
env = %env

85.3 µs ± 1.44 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [4]:
from IPython.display import display
#### Had to change display options to get this to print in full!
# pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 100000

## Import Initial Data

#### YOU'LL WANT TO CHANGE THESE LINKS IF YOU USE DIFFERENT DATA OR CHANGE LOCATION OF NOTEBOOK OR DATA

In [6]:
picks_dic = pd.read_csv('../../../SPE_006_originalData/OilSandsDB/PICKS_DIC.TXT',delimiter='\t')
picks = pd.read_csv('../../../SPE_006_originalData/OilSandsDB/PICKS.TXT',delimiter='\t')
wells = pd.read_csv('../../../SPE_006_originalData/OilSandsDB/WELLS.TXT',delimiter='\t')
gis = pd.read_csv('../../../well_lat_lng.csv')

In [7]:
picks.head()

,SitID,HorID,Pick,Quality
0,102496,1000,321,1
1,102496,2000,,-1
2,102496,3000,,-1
3,102496,4000,,-1
4,102496,5000,438,2


In [9]:
picks_dic

,HorID,Descriptopn
0,1000,mannville
1,2000,t61
2,3000,t51
3,4000,t41
4,5000,t31
5,6000,clw_wab
6,7000,t21
7,8000,e20
8,9000,t15
9,9500,e14


In [10]:
wells.head()

,SitID,UWI (AGS),UWI
0,102496,0674010812000,00/12-08-067-01W4/0
1,102497,0674020807000,00/07-08-067-02W4/0
2,102498,0674021109000,00/09-11-067-02W4/0
3,102500,0674022910000,00/10-29-067-02W4/0
4,102501,0674023406000,00/06-34-067-02W4/0


In [11]:
gis.head()

,SitID,UWI (AGS),UWI,HorID,Pick,Quality,lat,lng
0,102496,0674010812000,00/12-08-067-01W4/0,13000,475,3,54.785907,-110.129320
1,102497,0674020807000,00/07-08-067-02W4/0,13000,515,3,54.782284,-110.269446
2,102498,0674021109000,00/09-11-067-02W4/0,13000,480,3,54.785892,-110.186851
3,102500,0674022910000,00/10-29-067-02W4/0,13000,549,3,54.829624,-110.269422
4,102501,0674023406000,00/06-34-067-02W4/0,13000,529,2,54.840471,-110.224832


## Question 1: How many wells are included for each top? 

In [12]:
listOfTops = picks_dic.HorID.unique()

In [13]:
listOfTops

array([ 1000,  2000,  3000,  4000,  5000,  6000,  7000,  8000,  9000,
        9500, 10000, 11000, 12000, 13000, 14000])

### We'll use the fact that absent picks are categorized as -1 in terms of quality to exclude those and then count the rest that remain. You input files might require a different methodology!

In [39]:
noNullPicks = picks[picks.Quality != -1]
pick_counts = noNullPicks.groupby('HorID').SitID.count()

In [41]:
pick_counts

HorID
1000     1903
2000      517
3000      531
4000      597
5000     2188
6000      461
7000     2191
9000     2184
9500     2184
10000    2187
11000    2184
12000    2182
13000    2184
14000    2169
Name: SitID, dtype: int64

In [38]:
#### The total number of wells with any sort of pick is:
wells_with_picks_array = picks.SitID.unique()
print("number of wells with picks of some sort is: ",len(wells_with_picks_array))

number of wells with picks of some sort is:  2193


### A human decision is required to determine the minimum umber of tops needed to do anything with that top. For our purposes, we'll limit to those with at least 1900

### We're most interested in wells that have both the Top and Base McMurry picks, so let's see how many wells have both and get that list of wells.

In [81]:
topsMustHave = [13000,14000]

#### Idea for this task:
- Make a list of wells for each top in the topsMustHave list
- Find the wells that exist in all of the lists

In [94]:
def findWellsThatHaveCertainTop(top):
    #### Takes in top
    #### Returns a list of wells with the given top
    #print(top)
    rows_with_picks = picks[picks.Quality != -1]
    #print(rows_with_picks[0:4])
    rows_with_that_top = list(rows_with_picks.loc[rows_with_picks['HorID'] == top].SitID.unique())
    #print("before return",rows_with_that_top)
    return rows_with_that_top

In [95]:
def findWellsWithAllTopsGive(tops):
    #### Takes in a list of tops
    #### Returns a list of wells that include all of those tops. If only one top occurs, well is not included
    list_of_wells_with_tops = []
    for top in tops:
        list_of_wells_with_tops.append(findWellsThatHaveCertainTop(top))
    result = set(list_of_wells_with_tops[0])
    for s in list_of_wells_with_tops[1:]:
        result.intersection_update(s)
    return list(result)

In [96]:
wells_with_all_given_tops = findWellsWithAllTopsGive(topsMustHave)

In [99]:
len(wells_with_all_given_tops)

2164

# Import the logs and see how common each curve name is

# Make list of wells that includes both the minimum required curves & minimum required tops